In [ ]:
# Mutlti logreg sample

In [1]:
from collections import Counter
from sklearn.datasets import make_classification
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# summarize the dataset
print(X.shape, y.shape)

(1000, 10) (1000,)


In [2]:
X[:10]

array([[ 1.89149379, -0.39847585,  1.63856893,  0.01647165,  1.51892395,
        -3.52651223,  1.80998823,  0.58810926, -0.02542177, -0.52835426],
       [ 1.86913773, -0.56336215,  2.03411807,  0.38669445, -0.09584042,
        -2.92724017,  0.73921674, -0.55633   , -0.27340013,  0.72129251],
       [ 1.95259701, -2.83144572,  0.19055535, -0.66388697,  0.19159093,
        -1.8591214 , -0.10150205, -0.60957741, -2.07750191, -1.75469982],
       [-0.67042514, -0.69082031,  0.59354753, -0.06270367,  2.37357676,
        -0.52106039,  0.42872513, -0.26851085,  0.49226161, -1.34383965],
       [-1.39890137, -0.71522959, -0.6774507 ,  0.39153885, -0.45832391,
         2.17898372, -1.88550244, -0.96686613, -0.42588787, -0.20782615],
       [-2.94551044, -0.79851493, -1.92807818, -0.11749422,  3.75941261,
         2.55924496, -0.71375699,  0.87323395,  0.26641973, -3.52801155],
       [-0.99746413, -1.35338763, -1.50586396, -1.73258977,  3.66110299,
         0.44739896,  1.27005199,  0.76439032

In [3]:
y[:10]

array([1, 0, 1, 2, 0, 2, 2, 2, 0, 0])

In [4]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=5, n_classes=3, random_state=1)
# define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 0.681 (0.042)


In [5]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

# import eli5

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier

from numpy import array
from numpy import argmax
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [6]:
X, y = make_multilabel_classification(n_classes=3, random_state=0)

In [7]:
X

array([[3., 6., 1., ..., 1., 5., 0.],
       [3., 5., 5., ..., 1., 1., 1.],
       [3., 3., 5., ..., 0., 2., 1.],
       ...,
       [3., 7., 3., ..., 2., 4., 2.],
       [7., 2., 1., ..., 1., 1., 1.],
       [3., 5., 3., ..., 1., 2., 4.]])

In [8]:
y[:2]

array([[0, 1, 0],
       [0, 1, 0]])

## Data

In [9]:
RST = 0
df = pd.read_csv("./compare_manual_manuallVSsemiauto/train_manualVSsemiauto_rst_{}.csv".format(RST))
df_val = pd.read_csv("./compare_manual_manuallVSsemiauto/val_manual_only_rst_{}.csv".format(RST))
df_test = pd.read_csv("./compare_manual_manuallVSsemiauto/test_manual_only_rst_{}.csv".format(RST))

In [10]:
necessary_columns = list(df.columns)[1:] 
topics_list = necessary_columns + ['none']
topics_list

['offline_crime',
 'online_crime',
 'drugs',
 'gambling',
 'pornography',
 'prostitution',
 'slavery',
 'suicide',
 'terrorism',
 'weapons',
 'body_shaming',
 'health_shaming',
 'politics',
 'racism',
 'religion',
 'sexual_minorities',
 'sexism',
 'social_injustice',
 'none']

In [11]:
def get_labels(dataframe):
    labels =[]
    for i, el in dataframe.iterrows():
        current_sample_labels = []
        any_class = False
        for clm in necessary_columns:
            if el[clm] == 1:
                any_class = True
                current_sample_labels.append(clm)
        if any_class == False:
            current_sample_labels.append("none")
        current_sample_labels = ','.join(current_sample_labels)
        labels.append(current_sample_labels)
    return labels
train_labels = get_labels(df)
val_labels = get_labels(df_val)
test_labels = get_labels(df_test)

In [12]:
train_labels[:5]

['offline_crime', 'none', 'offline_crime', 'offline_crime', 'offline_crime']

In [13]:
df = pd.DataFrame({'text':list(df['text']), 'labels':train_labels})
df_val = pd.DataFrame({'text':list(df_val['text']), 'labels':val_labels})
df_test = pd.DataFrame({'text':list(df_test['text']), 'labels':test_labels})

In [14]:
mapping = dict()
mapping['none'] = 0

for label in train_labels:
    if label not in mapping:
        mapping[label] = len(mapping)

for label in test_labels:
    if label not in mapping:
        mapping[label] = len(mapping)
        
for label in val_labels:
    if label not in mapping:
        mapping[label] = len(mapping)

In [15]:
df['class'] = df['labels'].apply(lambda x: mapping[x])
df_test['class'] = df_test['labels'].apply(lambda x: mapping[x])
df_val['class'] = df_val['labels'].apply(lambda x: mapping[x])

In [16]:
df.head()

,text,labels,class
0,"Ман, можно же ведь и сабцы навернуть!А можно и...",offline_crime,1
1,Чтобы не было на них постороннего давления... ...,none,0
2,"С 12 лет на трое суток, а потом пять лет тюрьм...",offline_crime,1
3,там и смертная казнь есть за некоторые преступ...,offline_crime,1
4,Есть убивство во благо . И каждый настоящий че...,offline_crime,1


In [17]:
train_val = pd.concat([df,df_val])

In [18]:
train_val.head()

,text,labels,class
0,"Ман, можно же ведь и сабцы навернуть!А можно и...",offline_crime,1
1,Чтобы не было на них постороннего давления... ...,none,0
2,"С 12 лет на трое суток, а потом пять лет тюрьм...",offline_crime,1
3,там и смертная казнь есть за некоторые преступ...,offline_crime,1
4,Есть убивство во благо . И каждый настоящий че...,offline_crime,1


In [19]:
df_test.head()

,text,labels,class
0,Стреляют по ногам чтобы не убить а там артерия...,offline_crime,1
1,"Если тебе дали в морду, а ты пырнул ножом - до...",offline_crime,1
2,"Какие доказательства нужны, чтобы суд вынес об...",offline_crime,1
3,протокол составить на родителей. И выписать им...,offline_crime,1
4,Правда ли что коллекторы могут по законы силой...,offline_crime,1


In [20]:
onehot_encoder = OneHotEncoder(sparse=False)

In [21]:
trval_list = np.array(train_val['class'].tolist())
trval_list  = trval_list.reshape(-1, 1)
trval_list = onehot_encoder.fit_transform(trval_list)

In [22]:
test_list = np.array(df_test['class'].tolist())
test_list  = test_list.reshape(-1, 1)
test_list = onehot_encoder.fit_transform(test_list)

## Tf-idf logreg

In [23]:
# https://www.kaggle.com/kashnitsky/logistic-regression-tf-idf-baseline

In [24]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")
import re


import pymorphy2
morph = pymorphy2.MorphAnalyzer()

from pymystem3 import Mystem
from string import punctuation
from tqdm import tqdm
mystem = Mystem() 

def preprocess_text(text):
    
    text = re.sub("[^а-яА-Я]"," ",text)
    text = re.sub(" +"," ",text)
    text = text.split()

    tokens = [morph.parse(w)[0].normal_form for w in text]
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

preprocess_text("мама23663 мыла /!")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\N.Babakov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'мама мыло'

In [25]:
processed = []
for t in tqdm(train_val['text'].tolist()):
    pr = preprocess_text(t)
    processed.append(pr)
    
train_val['processed'] = processed

100%|███████████████████████████████████████████████████████████████████████████| 34542/34542 [02:33<00:00, 224.34it/s]


In [26]:
processed = []
for t in tqdm(df_test['text'].tolist()):
    pr = preprocess_text(t)
    processed.append(pr)
df_test['processed'] = processed

100%|█████████████████████████████████████████████████████████████████████████████| 1585/1585 [00:05<00:00, 310.61it/s]


In [27]:
target_vaiables_id2topic_dict = {val:key for key, val in mapping.items()}

In [30]:
def check_tfidf_onedim(sw = None, ngram_range=(1, 3), max_features=150000, text_label = 'text'):
    
    print("ngram_range {}, max_features {}".format(ngram_range, max_features))
    
    text_transformer = TfidfVectorizer(stop_words=sw, ngram_range=ngram_range, lowercase=True, max_features=max_features)
    
    X_train_text = text_transformer.fit_transform(train_val[text_label].tolist())
    X_test_text = text_transformer.transform(df_test[text_label])
    
    logit = LogisticRegression(multi_class='multinomial', solver='lbfgs',random_state=17, n_jobs=2)
    
#     cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
#     n_scores = cross_val_score(logit, X_train_text, train_val['class'], scoring='accuracy', cv=cv, n_jobs=-1)
#     print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
    
    logit.fit(X_train_text, train_val['class'])
    test_preds = logit.predict(X_test_text)
    metrics = compute_metrics(df_test['class'],test_preds)
    
    print(metrics)
    
#     return test_preds
 
res = check_tfidf_onedim(russian_stopwords, (1,2), 10000, 'processed')

ngram_range (1, 2), max_features 10000
{'accuracy': 0.4769716088328076, 'f1': 0.5347113331515103, 'precision': 0.7770162379784865, 'recall': 0.4821831303563374}


In [29]:
# def adjust_multilabel(y):
#     y_adjusted = []
#     for y_c in y:
#         y_test_curr = [0]*19
# #         print(y_c)
#         y_c = target_vaiables_id2topic_dict[np.argmax(y_c)]
# #         print(y_c)
#         for tag in y_c.split(","):
#             topic_index = topics_list.index(tag)
#             y_test_curr[topic_index] = 1
#         y_adjusted.append(y_test_curr)
# #         break
#     return y_adjusted

# adjust_multilabel(test_list)

def adjust_multilabel_onedim(y):
    y_adjusted = []
    for y_c in y:
        y_test_curr = [0]*19
        y_c = target_vaiables_id2topic_dict[y_c]
        for tag in y_c.split(","):
            topic_index = topics_list.index(tag)
            y_test_curr[topic_index] = 1
        y_adjusted.append(y_test_curr)
#         break
    return y_adjusted

def compute_metrics(labels, pred):
#     labels = adjust_multilabel(labels)
#     pred = adjust_multilabel(pred)
    
    labels = adjust_multilabel_onedim(labels)
    pred = adjust_multilabel_onedim(pred)
    
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred, average='weighted', zero_division = 0)
    acc = accuracy_score(labels, pred)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }
# compute_metrics(test_list,res)
# compute_metrics(df_test['class'].tolist(),res)

In [ ]:
# def check_tfidf(sw = None, ngram_range=(1, 3), max_features=150000, text_label = 'text'):
    
#     print("ngram_range {}, max_features {}".format(ngram_range, max_features))
    
#     text_transformer = TfidfVectorizer(stop_words=sw, ngram_range=ngram_range, lowercase=True, max_features=max_features)
#     X_train_text = text_transformer.fit_transform(train_val[text_label].tolist())
    
# #     print(X_train_text)
# #     raise Exception("STOPE")
    
#     X_test_text = text_transformer.transform(df_test[text_label])

# #     logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
    
#     logit = MultiOutputClassifier(estimator= LogisticRegression(random_state=17, n_jobs=-1)) 

# #     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

# #     cv_results = cross_val_score(logit, X_train_text, trval_list, cv=skf, scoring='f1_micro')

# #     print(cv_results, cv_results.mean())
    
#     logit.fit(X_train_text, trval_list)
#     test_preds = logit.predict(X_test_text)
#     metrics = compute_metrics(test_list,test_preds)
    
#     print(metrics)

    
# # check_tfidf()   
# check_tfidf(russian_stopwords, (1,2), 10000, 'processed')

In [ ]:
for ngram in [(1,3),(1,4)]:
    for feat in [100000, 300000]:
        check_tfidf_onedim(russian_stopwords, ngram, feat, 'processed')    
        print("="*100)

ngram_range (1, 3), max_features 100000


In [ ]:
from sklearn.svm import SVC

In [ ]:
def check_SVM(ngram_range=(1, 3), max_features=150000, kernel = 'linear'):
    
    print("ngram_range {}, max_features {}".format(ngram_range, max_features))
    
    text_transformer = TfidfVectorizer(stop_words=russian_stopwords, ngram_range=ngram_range, lowercase=True, max_features=max_features)
    X_train_text = text_transformer.fit_transform(train_val['processed'].tolist())
    
    X_test_text = text_transformer.transform(df_test['processed'])

#     logit = LogisticRegression(C=5e1, solver='lbfgs', multi_class='multinomial', random_state=17, n_jobs=4)
    logit = SVC(kernel = kernel)
    print("fitting ...")
    logit.fit(X_train_text, train_val['inappropriate'])
    test_preds = logit.predict(X_test_text)
    print(classification_report(df_test['inappropriate'], test_preds))
    
    check_SVM
    
check_SVM()